# ***Sentiment Analysis on Moroccan Arabic Dialect: A Lexicon Approach***

### Unsupervised or The Lexicon-based approach uses pre-prepared sentiment lexicon to score a document by aggregating the sentiment scores of all the words in the document. With this approach a dictionary of positive and negative words is required, with a positive or negative sentiment value assigned to each of the words. Different approaches to creating dictionaries have been proposed, including manual and automatic approaches.

## **Importing the required libraries**

### In our work, we have decided to apply a lexicon-based approach in order to avoid the need to generate the labelled training set which is a bit small to get a great classification score (in our second notebook you will find the supervised method with the training steps). 
### the fact that a lexicon-based approach can be more easily understood and modified by a human is considered a significant advantage for our work.
### In lexicon-based classification, data is assigned labels by comparing the number of words that appear from two opposed lexicons, such as positive and negative sentiment. Creating such words lists is often easier than labeling instances, Each word has a polarity (1 or 0 for positive or negative respectively).
### The dictionaries could be built manually or automatically, using seed words to expands the list of words.


In [1]:
!pip install pyarabic

In [2]:
from os import listdir
from os.path import isfile, join
import re #for regular expressions
import pandas as pd 
pd.set_option("display.max_colwidth", 200)
import numpy as np 
#import matplotlib.pyplot as plt 
#import seaborn as sns
#import string
import nltk # for text manipulation
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from matplotlib import pyplot as plt
#from aaransia import transliterate, SourceLanguageError
import codecs
import unicodedata as ud
#from tashaphyne.stemming import ArabicLightStemmer
import pyarabic.araby as araby
from textblob import TextBlob
from nltk.corpus import stopwords

from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score


In [3]:
data = pd.read_csv("../input/sentiment-analysis-on-moroccan-arabic-dialect/test_stage1.csv")
data

,ID,comment
0,1,"التلقيح نعمة لانه فابور,الناس في مصر را كتخلص عليه"
1,2,أثبت التلقيح أهميته في الحد من الوفيات
2,3,انا ملقح جوج مرات والحمد لله عندي المناعة ديالي حيت درت التلقيح ومعندي حتى مشكل ومتشكر المسؤولين وملكنا الحبيب
3,4,كنــا متأكديــن من أن جلالـة الملــك سيجعل التلقيـح مجاني لكل المغاربة وحتى الأجانب ، وكنا متأكدين أنه أول من سيقوم يالتلقيح ، ليس لتحفيزنا ، وزرع الثقة ، لكن بوصفه قائدنا الأول ، و الساهر على سلا...
4,5,شعب أناني مشبع بثقافة الخرافة والمؤامرة… بنادم عندو هانية يضر ملايين آخرين باش يرضي الغرور ديالو ونزعة المؤامرة اللي مزروعة ليه فالمخ بحال يلا تصنع لينا اليورانيوم المخصب غادي يستهدفوه باش يقضيوا ...
...,...,...
235,236,\nحنا المشكل ديالنا هما الفيروسات البشرية أما الفيروسات الأخرى ولله الحمد لدينا مناعة ربانية تحمينا حنا راه ماشي قطيع ديال الأغنام باش كتقولوا مناعة جماعية سيرو ديرو التلقيح الأصحاب الرضاعة اللي ك...
236,237,\nالسيد بلفقيه، اسمحوا لي بسؤال : هل الملقحون يصابون بكوفيد19 ؟؟؟ هل ينقلون العدوى للآخرين؟؟؟\nحسب ما حدث في كل بقاع العالم من انتشار المتحور دلتا، تأكد أن الملقحين يصابون و ينقلون العدوى…..\nالسؤ...
237,238,اللقاح هدفه هو تقليص عدد سكان العالم لأنه أصبح جد دروري الوصول لهدفهم سيكون عبر تحديد النسل في الأخير هدا الوباء هو إعلامي و إقتصادي أكتر من صحي
238,239,\nاخوتي في الله انه و بعد يقين في الله قررت ان لا اتلقى اي لقاح كما هي الابقار لها حرية اختيار المسلك الدي ترتضيه الى مصيرها المحتوم … و لكن و بعد الحاح من طرف كل أفراد العائلة قررت ان أتلقى اللقا...


## **Data preprocessing:**

#### Text data is one of the most unstructured forms of available data and when comes to deal with Human language then it’s too complex, it contains noise in various forms like emotions, punctuation, text in a different case.
#### Cleaning or preprocessing the data is as important as model building it is used to clean the text data, make it predictable and analyzable so it can be given to the model.
#### There are different ways to preprocess the Data:

### **Stop Words**

#### NLTK library is a common library that is used to remove stop words and include approximately 243 stop words which it removes.
#### I created a new set of words that contains 226 stop words in Moroccan dialect.

In [4]:
def get_stops():
    with codecs.open("../input/lexicon-based-approach-data/stop_words_Maroc.txt", encoding="utf-8", errors="ignore") as myfile:
         get_stops=myfile.read()
    get_stops=get_stops.split()
    return get_stops

In [5]:
stops = set(stopwords.words("arabic"))
len(stops)

701

#### I create a function ‘remove_stopwords’ that's going to remove the Stop Words which do not contain important significance to be used in sentiment analysis.

In [6]:
def remove_stop_words(text):
    zen = TextBlob(text)
    words = zen.words
    return " ".join([w for w in words if not w in stops and not w in get_stops()  and len(w) >= 2])

### **Normalization**

#### a. Remove any punctuation from the string such as (. “” ; ').
#### b. Remove emojis.
#### c. Remove Latin language.
#### d. Delete multispacer.
#### e. Delete '...' and replace it with space.
#### f. Replace multiple letters with a single one except ا, و, ي, ه .
#### g. Replace more than 3 “ه” (a laugh) with “ههه ”.
#### h. Delete numbers.
#### g. Remove non-letters from the word such as (+ = ~ $).
#### h. Replace “إ”, “أ”, and “ آ” with bare alif " ا" regardless of position in the word.
#### k. Replace “گ” with “ك”.

In [7]:
def process(text):
    text = araby.strip_tashkeel(text) #delete *tashkil

    text = re.sub('\ـ+', ' ', text)  # delete letter madda
    text= re.sub(r'\...',' ',text)
    text= re.sub(r'[a-z]','',text)
    text= re.sub(r'[A-Z]','',text)
    text= re.sub(r'[0-9]','',text)
    text= re.sub(r'_',' ',text)
    text= re.sub(r'[^\w\s]','',text) # remove imojies
    text = re.sub('\ر+', 'ر', text)  # duplicate ra2
    text = re.sub('\اا+','ا',text)     #duplicate alif
    text = re.sub('\ووو+','و',text)  #duplicate waw (more than 3 times goes to 1
    text = re.sub('\ف+','ف',text)  #duplicate fa2
    text = re.sub('\ههه+','ههه',text)  #duplicate ha2 (more than 3 times goes to 1
    text = re.sub('\ة+','ة',text)
    text = re.sub('\ييي+','ي',text)
    text = re.sub('أ','ا',text) # after to avoid mixing
    text = re.sub('آ','ا',text) # after to avoid mixing
    text = re.sub('إ','ا',text) # after to avoid mixing
    #text = re.sub('ة','ه',text) # after ةة to avoid mixing ههه
    #text = re.sub('ى','ي',text)
    text = re.sub("گ", "ك", text)
    text = re.sub('ي+','ي',text)
    text = remove_stop_words(text) #remove stopwords
    text = " ".join(text.split()) #delete multispace
    return text

In [8]:
data['cleaning_process']=data['comment']
for i in range(len(data['comment'])):
    data['cleaning_process'].iloc[i]=process(data['comment'].iloc[i])

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [9]:
data

,ID,comment,cleaning_process
0,1,"التلقيح نعمة لانه فابور,الناس في مصر را كتخلص عليه",التلقيح نعمة فابورالناس مصر كتخلص
1,2,أثبت التلقيح أهميته في الحد من الوفيات,اثبت التلقيح اهميته الحد الوفيات
2,3,انا ملقح جوج مرات والحمد لله عندي المناعة ديالي حيت درت التلقيح ومعندي حتى مشكل ومتشكر المسؤولين وملكنا الحبيب,ملقح جوج مرات والحمد عندي المناعة ديالي درت التلقيح ومعندي مشكل ومتشكر المسؤولين وملكنا الحبيب
3,4,كنــا متأكديــن من أن جلالـة الملــك سيجعل التلقيـح مجاني لكل المغاربة وحتى الأجانب ، وكنا متأكدين أنه أول من سيقوم يالتلقيح ، ليس لتحفيزنا ، وزرع الثقة ، لكن بوصفه قائدنا الأول ، و الساهر على سلا...,متاكدي جلال المل سيجعل التلقي مجاني المغاربة وحتى الاجانب وكنا متاكدين انه سيقوم يالتلقيح لتحفيزنا وزرع الثقة بوصفه قائدنا الاول الساهر سلامتنا وامنن نتمنى صحة وعافية وطول العم وننتظر يصل دورنا اخ...
4,5,شعب أناني مشبع بثقافة الخرافة والمؤامرة… بنادم عندو هانية يضر ملايين آخرين باش يرضي الغرور ديالو ونزعة المؤامرة اللي مزروعة ليه فالمخ بحال يلا تصنع لينا اليورانيوم المخصب غادي يستهدفوه باش يقضيوا ...,شعب اناني مشبع بثقافة الخرافة والمؤامرة بنادم هانية يضر ملاين اخرين يرضي الغرور ونزعة المؤامرة مزروعة فالمخ يلا تصنع اليورانيوم المخصب غادي يستهدفوه يقضيوا باللقاح الله ملاير البشر لقحات فالصين وا...
...,...,...,...
235,236,\nحنا المشكل ديالنا هما الفيروسات البشرية أما الفيروسات الأخرى ولله الحمد لدينا مناعة ربانية تحمينا حنا راه ماشي قطيع ديال الأغنام باش كتقولوا مناعة جماعية سيرو ديرو التلقيح الأصحاب الرضاعة اللي ك...,المشكل الفيروسات البشرية الفيروسات الاخرى ولله الحمد لدينا مناعة ربانية تحمينا قطيع الاغنام كتقولوا مناعة جماعية سيرو ديرو التلقيح الاصحاب الرضاعة كيمصوا ثروات الفقراء خليهم يعيشوا الفقراء تحدانا ...
236,237,\nالسيد بلفقيه، اسمحوا لي بسؤال : هل الملقحون يصابون بكوفيد19 ؟؟؟ هل ينقلون العدوى للآخرين؟؟؟\nحسب ما حدث في كل بقاع العالم من انتشار المتحور دلتا، تأكد أن الملقحين يصابون و ينقلون العدوى…..\nالسؤ...,السيد بلفقيه اسمحوا بسؤال الملقحون يصابون بكوفيد ينقلون العدوى للاخرين حدث بقاع العالم انتشار المتحور دلتا تاكد الملقحين يصابون ينقلون العدوى السؤال الهدف التلقيح اذا الملقح مثله الملقح يصابان ينق...
237,238,اللقاح هدفه هو تقليص عدد سكان العالم لأنه أصبح جد دروري الوصول لهدفهم سيكون عبر تحديد النسل في الأخير هدا الوباء هو إعلامي و إقتصادي أكتر من صحي,اللقاح هدفه تقليص عدد سكان العالم اصبح جد دروري الوصول لهدفهم سيكون تحديد النسل الاخير الوباء اعلامي اقتصادي صحي
238,239,\nاخوتي في الله انه و بعد يقين في الله قررت ان لا اتلقى اي لقاح كما هي الابقار لها حرية اختيار المسلك الدي ترتضيه الى مصيرها المحتوم … و لكن و بعد الحاح من طرف كل أفراد العائلة قررت ان أتلقى اللقا...,اخوتي الله انه يقين الله قرت اتلقى اي لقاح الابقار حرية اختيار المسلك ترتضيه مصيرها المحتوم الحاح طرف افراد العائلة قرت اتلقى اللقاح اخدت الجرعة الاولى مر نصف شهر دلك اللقاح اوكسفورد اي استرازينكا...


### **Stemming**

#### I used ISRI, it is a root-extraction stemmer without a root dictionary, which serves as a light stemmer. Light stemming means removing diacritics represented by short vowels, stop words, punctuation, numbers, the definite article, prefixes, and suffixes .
#### The characteristics of this stemmer are:
#### 1) it is implemented in python and integrated into NLTK which can be invoked easily
#### 2) it is well documented in the NLTK documentation.

In [10]:
import argparse

from nltk.stem.isri import ISRIStemmer
import sys

In [11]:
def light_stem(text):
    words = text.split()
    result = list()
    stemmer = ISRIStemmer()
    for word in words:
        word = stemmer.norm(word, num=1)      # remove diacritics which representing Arabic short vowels
        if not word in stemmer.stop_words:    # exclude stop words from being processed
            word = stemmer.pre32(word)        # remove length three and length two prefixes in this order
            word = stemmer.suf32(word)        # remove length three and length two suffixes in this order
            word = stemmer.waw(word)          # remove connective ‘و’ if it precedes a word beginning with ‘و’
            word = stemmer.norm(word, num=2)  # normalize initial hamza to bare alif
        result.append(word)
    return ' '.join(result)

In [12]:
data['stemming']=data['comment']

In [13]:
for i in range(len(data['comment'])):
    data['stemming'].iloc[i]=light_stem(data['cleaning_process'].iloc[i])
            
               

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [14]:
data

,ID,comment,cleaning_process,stemming
0,1,"التلقيح نعمة لانه فابور,الناس في مصر را كتخلص عليه",التلقيح نعمة فابورالناس مصر كتخلص,تلقيح نعمة فابورالناس مصر كتخلص
1,2,أثبت التلقيح أهميته في الحد من الوفيات,اثبت التلقيح اهميته الحد الوفيات,اثبت تلقيح اهميته الحد وفي
2,3,انا ملقح جوج مرات والحمد لله عندي المناعة ديالي حيت درت التلقيح ومعندي حتى مشكل ومتشكر المسؤولين وملكنا الحبيب,ملقح جوج مرات والحمد عندي المناعة ديالي درت التلقيح ومعندي مشكل ومتشكر المسؤولين وملكنا الحبيب,ملقح جوج مرات حمد عندي مناعة ديالي درت تلقيح ومعندي مشكل ومتشكر مسؤول وملك حبيب
3,4,كنــا متأكديــن من أن جلالـة الملــك سيجعل التلقيـح مجاني لكل المغاربة وحتى الأجانب ، وكنا متأكدين أنه أول من سيقوم يالتلقيح ، ليس لتحفيزنا ، وزرع الثقة ، لكن بوصفه قائدنا الأول ، و الساهر على سلا...,متاكدي جلال المل سيجعل التلقي مجاني المغاربة وحتى الاجانب وكنا متاكدين انه سيقوم يالتلقيح لتحفيزنا وزرع الثقة بوصفه قائدنا الاول الساهر سلامتنا وامنن نتمنى صحة وعافية وطول العم وننتظر يصل دورنا اخ...,متاكدي جلال المل سيجعل تلقي مجا مغاربة وحتى اجانب وكنا متاكد انه سيقوم يالتلقيح لتحفيز وزرع ثقة بوصفه قائد اول ساهر سلامت وامنن نتمنى صحة وعافية وطول العم وننتظر يصل دور اخذ لقاح
4,5,شعب أناني مشبع بثقافة الخرافة والمؤامرة… بنادم عندو هانية يضر ملايين آخرين باش يرضي الغرور ديالو ونزعة المؤامرة اللي مزروعة ليه فالمخ بحال يلا تصنع لينا اليورانيوم المخصب غادي يستهدفوه باش يقضيوا ...,شعب اناني مشبع بثقافة الخرافة والمؤامرة بنادم هانية يضر ملاين اخرين يرضي الغرور ونزعة المؤامرة مزروعة فالمخ يلا تصنع اليورانيوم المخصب غادي يستهدفوه يقضيوا باللقاح الله ملاير البشر لقحات فالصين وا...,شعب انا مشبع بثقافة خرافة مؤامرة بنادم هانية يضر ملا اخر يرضي غرور ونزعة مؤامرة مزروعة فالمخ يلا تصنع يورانيوم مخصب غادي يستهدفوه يقضي لقاح الله ملاير بشر لقح فالص هند ودول فايتة بلي دالساكنة مشفن...
...,...,...,...,...
235,236,\nحنا المشكل ديالنا هما الفيروسات البشرية أما الفيروسات الأخرى ولله الحمد لدينا مناعة ربانية تحمينا حنا راه ماشي قطيع ديال الأغنام باش كتقولوا مناعة جماعية سيرو ديرو التلقيح الأصحاب الرضاعة اللي ك...,المشكل الفيروسات البشرية الفيروسات الاخرى ولله الحمد لدينا مناعة ربانية تحمينا قطيع الاغنام كتقولوا مناعة جماعية سيرو ديرو التلقيح الاصحاب الرضاعة كيمصوا ثروات الفقراء خليهم يعيشوا الفقراء تحدانا ...,مشكل فيروس بشرية فيروس اخرى ولله حمد لدي مناعة ربانية تحمي قطيع اغنام كتقول مناعة جماعية سيرو ديرو تلقيح اصحاب رضاعة كيمص ثرو فقراء خلي يعيش فقراء تحدا سيمشي رجعة تخربيق صافي لجنة علمية يريد تحكم ...
236,237,\nالسيد بلفقيه، اسمحوا لي بسؤال : هل الملقحون يصابون بكوفيد19 ؟؟؟ هل ينقلون العدوى للآخرين؟؟؟\nحسب ما حدث في كل بقاع العالم من انتشار المتحور دلتا، تأكد أن الملقحين يصابون و ينقلون العدوى…..\nالسؤ...,السيد بلفقيه اسمحوا بسؤال الملقحون يصابون بكوفيد ينقلون العدوى للاخرين حدث بقاع العالم انتشار المتحور دلتا تاكد الملقحين يصابون ينقلون العدوى السؤال الهدف التلقيح اذا الملقح مثله الملقح يصابان ينق...,سيد بلفقيه اسمح بسؤال ملقح يصاب بكوفيد ينقل عدوى اخر حدث بقاع عالم انتشار متحور دلتا تاكد ملقح يصاب ينقل عدوى سؤال هدف تلقيح اذا ملقح مثله ملقح يصاب ينقل عدوى نرغم ناس تلقيح فهذه مسالة مواز قوى مو...
237,238,اللقاح هدفه هو تقليص عدد سكان العالم لأنه أصبح جد دروري الوصول لهدفهم سيكون عبر تحديد النسل في الأخير هدا الوباء هو إعلامي و إقتصادي أكتر من صحي,اللقاح هدفه تقليص عدد سكان العالم اصبح جد دروري الوصول لهدفهم سيكون تحديد النسل الاخير الوباء اعلامي اقتصادي صحي,لقاح هدفه تقليص عدد سكان عالم اصبح جد دروري وصول لهدف سيك تحديد نسل اخير وباء اعلامي اقتصادي صحي
238,239,\nاخوتي في الله انه و بعد يقين في الله قررت ان لا اتلقى اي لقاح كما هي الابقار لها حرية اختيار المسلك الدي ترتضيه الى مصيرها المحتوم … و لكن و بعد الحاح من طرف كل أفراد العائلة قررت ان أتلقى اللقا...,اخوتي الله انه يقين الله قرت اتلقى اي لقاح الابقار حرية اختيار المسلك ترتضيه مصيرها المحتوم الحاح طرف افراد العائلة قرت اتلقى اللقاح اخدت الجرعة الاولى مر نصف شهر دلك اللقاح اوكسفورد اي استرازينكا...,اخوتي الله انه يقين الله قرت اتلقى اي لقاح ابقار حرية اختيار مسلك ترتضيه مصير محتوم حاح طرف افراد عائلة قرت اتلقى لقاح اخدت جرعة اولى مر نصف شهر دلك لقاح اوكسفورد اي استرازينكا اول لاخدي قاح اعا ا...


## **Model Creation: LEXICON BASED APPROACH**

#### Since there is a lack in the Moroccan dialect word dictionaries, I built my own one. I created a function that return the number of positive and negative words based on the dictionary I created, it contains 712 “positive” and 1200 “negative”.

In [15]:
def get_corpus():
    with codecs.open("../input/lexicon-based-approach-data/positive_version91.txt", encoding="utf-8", errors="ignore") as myfile:
         pos_terms=myfile.read()
    pos_terms=pos_terms.split()
    
    with codecs.open("../input/lexicon-based-approach-data/negative_version91.txt", encoding="utf-8" , errors="ignore") as myfie:
        neg_terms=myfie.read()
    neg_terms=neg_terms.split()
    return pos_terms,neg_terms

#### Then, I created a function that calculate the subtraction between the positive score and the negative score,
#### If the comment contains more positive word then it is obviously positive, otherwise it is negative.

In [16]:
def text_to_valeur_sentiment(text):
    i=0
    j=0
    text_words = []
    words = text.split()
    positive_terms,neg_terms=get_corpus()
    for word in words:
        if word in positive_terms :
            
            i=i+1
        if word in neg_terms :
            
            j=j+1
    if (i-j)>0:
        return  1
    if (i-j)<0:
        return  0
    if (i-j)==0:
        return 1

In [17]:
def text_to_score(text):
    i=0
    j=0
    text_words = []
    words = text.split()
    positive_terms,neg_terms=get_corpus()
    for word in words:
        if word in positive_terms :
            
            i=i+1
        if word in neg_terms :
            
            j=j+1
    return i,j

In [18]:
text_to_score('الحمدلله عنيا دمعو بالفرحة')[0]

0

In [19]:
data['pos_score']=data['comment']
data['neg_score']=data['comment']

In [20]:
for i in range(len(data['comment'])):
    data['pos_score'].iloc[i]=text_to_score(data['stemming'].iloc[i])[0]
for i in range(len(data['comment'])):
    data['neg_score'].iloc[i]=text_to_score(data['stemming'].iloc[i])[1]

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [21]:
data['label_lexicon']=data['comment']

In [22]:
for i in range(len(data['comment'])):
    data['label_lexicon'].iloc[i]=text_to_valeur_sentiment(data['stemming'].iloc[i])

In [23]:
data_top = data.head() 
# display 
data_top

,ID,comment,cleaning_process,stemming,pos_score,neg_score,label_lexicon
0,1,"التلقيح نعمة لانه فابور,الناس في مصر را كتخلص عليه",التلقيح نعمة فابورالناس مصر كتخلص,تلقيح نعمة فابورالناس مصر كتخلص,1,0,1
1,2,أثبت التلقيح أهميته في الحد من الوفيات,اثبت التلقيح اهميته الحد الوفيات,اثبت تلقيح اهميته الحد وفي,1,1,1
2,3,انا ملقح جوج مرات والحمد لله عندي المناعة ديالي حيت درت التلقيح ومعندي حتى مشكل ومتشكر المسؤولين وملكنا الحبيب,ملقح جوج مرات والحمد عندي المناعة ديالي درت التلقيح ومعندي مشكل ومتشكر المسؤولين وملكنا الحبيب,ملقح جوج مرات حمد عندي مناعة ديالي درت تلقيح ومعندي مشكل ومتشكر مسؤول وملك حبيب,4,1,1
3,4,كنــا متأكديــن من أن جلالـة الملــك سيجعل التلقيـح مجاني لكل المغاربة وحتى الأجانب ، وكنا متأكدين أنه أول من سيقوم يالتلقيح ، ليس لتحفيزنا ، وزرع الثقة ، لكن بوصفه قائدنا الأول ، و الساهر على سلا...,متاكدي جلال المل سيجعل التلقي مجاني المغاربة وحتى الاجانب وكنا متاكدين انه سيقوم يالتلقيح لتحفيزنا وزرع الثقة بوصفه قائدنا الاول الساهر سلامتنا وامنن نتمنى صحة وعافية وطول العم وننتظر يصل دورنا اخ...,متاكدي جلال المل سيجعل تلقي مجا مغاربة وحتى اجانب وكنا متاكد انه سيقوم يالتلقيح لتحفيز وزرع ثقة بوصفه قائد اول ساهر سلامت وامنن نتمنى صحة وعافية وطول العم وننتظر يصل دور اخذ لقاح,2,1,1
4,5,شعب أناني مشبع بثقافة الخرافة والمؤامرة… بنادم عندو هانية يضر ملايين آخرين باش يرضي الغرور ديالو ونزعة المؤامرة اللي مزروعة ليه فالمخ بحال يلا تصنع لينا اليورانيوم المخصب غادي يستهدفوه باش يقضيوا ...,شعب اناني مشبع بثقافة الخرافة والمؤامرة بنادم هانية يضر ملاين اخرين يرضي الغرور ونزعة المؤامرة مزروعة فالمخ يلا تصنع اليورانيوم المخصب غادي يستهدفوه يقضيوا باللقاح الله ملاير البشر لقحات فالصين وا...,شعب انا مشبع بثقافة خرافة مؤامرة بنادم هانية يضر ملا اخر يرضي غرور ونزعة مؤامرة مزروعة فالمخ يلا تصنع يورانيوم مخصب غادي يستهدفوه يقضي لقاح الله ملاير بشر لقح فالص هند ودول فايتة بلي دالساكنة مشفن...,6,3,1


In [24]:

finaldata = pd.read_csv("../input/test-satge2/test_stage2_with_all columns.csv" )
y2 = finaldata['label']


In [25]:
ID1=[]
label_lexicon1=[]
for i in range (240):
    ID1.append(data['ID'].iloc[i])
    label_lexicon1.append(data['label_lexicon'].iloc[i])


import pandas as pd
d = {
    'ID':ID1,
    'label': label_lexicon1
}

df = pd.DataFrame(d, columns = ['ID','label'])
df.to_csv('PredictedLabels.csv', index=False)

In [26]:
#test the score of the file(compare the predicted labeled with the fake one)
#similar to kaggle scoring

finaldata = pd.read_csv("../input/test-satge2/test_stage2_with_all columns.csv" )
y2 = finaldata['label']

finaldata = pd.read_csv("PredictedLabels.csv" )
y4 = finaldata['label']

print("Accuracy:",metrics.accuracy_score(y2, y4)*100)
print("Precision:",precision_score(y2, y4,average='weighted')*100)
print("Recall:",recall_score(y2, y4,average='micro')*100)
print("Classification Report: ")
print(classification_report(y2, y4))

Accuracy: 91.66666666666666
Precision: 91.67824395665463
Recall: 91.66666666666666
Classification Report: 
              precision    recall  f1-score   support

           0       0.92      0.91      0.92       120
           1       0.91      0.93      0.92       120

    accuracy                           0.92       240
   macro avg       0.92      0.92      0.92       240
weighted avg       0.92      0.92      0.92       240

